In [1]:
import itertools
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import DraperQFTAdder

In [ ]:
a = 1
b = 2

In [70]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import QFT
from qiskit_aer import AerSimulator
import numpy as np

NUMBITS = 5

# Convert inputs to bit arrays (LSB first)
a_bit = [int(i) for i in bin(a)[2:].zfill(NUMBITS)][::-1]
b_bit = [int(i) for i in bin(b)[2:].zfill(NUMBITS)][::-1]

# Quantum and classical registers
a_reg = QuantumRegister(NUMBITS, "a")
b_reg = QuantumRegister(NUMBITS, "b")
c_reg = ClassicalRegister(NUMBITS, "c")
qc = QuantumCircuit(a_reg, b_reg, c_reg)

# Initialize a and b
for i, bit in enumerate(a_bit):
    if bit == 1:
        qc.x(a_reg[i])
for i, bit in enumerate(b_bit):
    if bit == 1:
        qc.x(b_reg[i])

# Apply QFT to a
qc.append(QFT(NUMBITS, do_swaps=False), a_reg)

# Add b into a using controlled phase gates
for i in range(NUMBITS):
    for j in range(NUMBITS):
        if j <= i:
            angle = (2 * np.pi) / (2 ** (i - j + 1))
            qc.cp(angle, b_reg[j], a_reg[i])

# Apply inverse QFT
qc.append(QFT(NUMBITS, do_swaps=False).inverse(), a_reg)

# Measure result in a
qc.measure(a_reg, c_reg)

# Run simulation
backend = AerSimulator()
job = backend.run(transpile(qc, backend), shots=1024)
counts = job.result().get_counts()

# Extract most likely result
most_common = max(counts, key=counts.get)
result = int(most_common, 2)

print(f"Result: {result}")

Result: 3


In [72]:
from qiskit import QuantumCircuit, QuantumRegister

# Create an initial quantum circuit with 2 qubits
qc = QuantumCircuit(2)

# Create a new quantum register with 3 more qubits
new_qreg = QuantumRegister(3, name='q2')

# Add the new register to the existing circuit
qc.add_register(new_qreg)

# The circuit now has 5 qubits: 2 from the original, 3 from the new register
print(qc)

      
 q_0: 
      
 q_1: 
      
q2_0: 
      
q2_1: 
      
q2_2: 
      


In [71]:
qc.draw()

┌───┐┌──────┐                                                         »
a_0: ┤ X ├┤0     ├─■───────────────────────────────────────────────────────»
     └───┘│      │ │                                                       »
a_1: ─────┤1     ├─┼──────■────────■───────────────────────────────────────»
          │      │ │      │        │                                       »
a_2: ─────┤2 QFT ├─┼──────┼────────┼──────■────────■─────────────────■─────»
          │      │ │      │        │      │        │                 │     »
a_3: ─────┤3     ├─┼──────┼────────┼──────┼────────┼────────■────────┼─────»
          │      │ │      │        │      │        │        │        │     »
a_4: ─────┤4     ├─┼──────┼────────┼──────┼────────┼────────┼────────┼─────»
          └──────┘ │P(π)  │P(π/2)  │      │P(π/4)  │        │P(π/8)  │     »
b_0: ──────────────■──────■────────┼──────■────────┼────────■────────┼─────»
     ┌───┐                         │P(π)           │P(π/2)           │     »
b_1: ┤ X ├─────────────────────────■───────────────■─────────────────┼─────»
     └───┘                                                           │P(π) »
b_2: ────────────────────────────────────────────────────────────────■─────»
                                                                           »
b_3: ──────────────────────────────────────────────────────────────────────»
                                                                           »
b_4: ──────────────────────────────────────────────────────────────────────»
                                                                           »
c: 5/══════════════════════════════════════════════════════════════════════»
                                                                           »
«                                                                          »
«a_0: ─────────────────────────────────────────────────────────────────────»
«                                                                          »
«a_1: ─────────────────────────────────────────────────────────────────────»
«                                                                          »
«a_2: ─────────────────────────────────────────────────────────────────────»
«                                                                          »
«a_3: ─■──────────────────■─────────────────■──────────────────────────────»
«      │                  │                 │                              »
«a_4: ─┼────────■─────────┼────────■────────┼──────■────────■────────■─────»
«      │        │P(π/16)  │        │        │      │        │        │     »
«b_0: ─┼────────■─────────┼────────┼────────┼──────┼────────┼────────┼─────»
«      │P(π/4)            │        │P(π/8)  │      │        │        │     »
«b_1: ─■──────────────────┼────────■────────┼──────┼────────┼────────┼─────»
«                         │P(π/2)           │      │P(π/4)  │        │     »
«b_2: ────────────────────■─────────────────┼──────■────────┼────────┼─────»
«                                           │P(π)           │P(π/2)  │     »
«b_3: ──────────────────────────────────────■───────────────■────────┼─────»
«                                                                    │P(π) »
«b_4: ───────────────────────────────────────────────────────────────■─────»
«                                                                          »
«c: 5/═════════════════════════════════════════════════════════════════════»
«                                                                          »
«     ┌───────┐┌─┐            
«a_0: ┤0      ├┤M├────────────
«     │       │└╥┘┌─┐         
«a_1: ┤1      ├─╫─┤M├─────────
«     │       │ ║ └╥┘┌─┐      
«a_2: ┤2 IQFT ├─╫──╫─┤M├──────
«     │       │ ║  ║ └╥┘┌─┐   
«a_3: ┤3      ├─╫──╫──╫─┤M├───
«     │       │ ║  ║  ║ └╥┘┌─┐
«a_4: ┤4      ├─╫──╫──╫──╫─┤M├
«     └───────┘ ║  ║  ║  ║ └╥┘
«b_0: ──────────╫──╫──╫──╫──╫─
«               ║  ║  ║  ║  ║ 
«b_1: ──────────╫──╫──╫──╫──╫─
«               ║  ║  ║  ║  ║

In [29]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import QFT
from qiskit_aer import AerSimulator
import numpy as np

NUMBITS = 5

a = 12  # Quantum input (signed integer)
b = 1   # Classical input (signed integer)

# Validate that a and b are within representable range for NUMBITS
MIN_VAL = -2**(NUMBITS - 1)
MAX_VAL = 2**(NUMBITS - 1) - 1

if not (MIN_VAL <= a <= MAX_VAL):
    raise ValueError(f"Value of 'a' = {a} is out of range for {NUMBITS} bits (must be between {MIN_VAL} and {MAX_VAL})")
if not (MIN_VAL <= b <= MAX_VAL):
    raise ValueError(f"Value of 'b' = {b} is out of range for {NUMBITS} bits (must be between {MIN_VAL} and {MAX_VAL})")

# Helper to convert signed int to 2's complement binary (LSB first)
def int_to_twos_complement(n, bits):
    if n < 0:
        n = (1 << bits) + n
    return [int(x) for x in bin(n)[2:].zfill(bits)][::-1]

# Bit arrays in 2's complement
a_bin = int_to_twos_complement(a, NUMBITS)
b_bin = int_to_twos_complement(b, NUMBITS)

print(f"Input a (quantum):   {a} -> 2's comp bits (LSB first): {a_bin}")
print(f"Input b (classical): {b} -> 2's comp bits (LSB first): {b_bin}")

# Set up circuit
qreg = QuantumRegister(NUMBITS, "q")
creg = ClassicalRegister(NUMBITS, "c")
qc = QuantumCircuit(qreg, creg)

# Initialize |a⟩ in the quantum register
for i, bit in enumerate(a_bin):
    if bit == 1:
        qc.x(qreg[i])

# Apply QFT
qc.append(QFT(num_qubits=NUMBITS, do_swaps=False), qreg)

# Add classical value b (2's complement) via controlled phase rotations
b_int = int(''.join(str(x) for x in b_bin[::-1]), 2)
if b >= 0:
    b_val = b_int
else:
    b_val = b_int - (1 << NUMBITS)  # Signed integer interpretation

for j in range(NUMBITS):
    angle = (b_val * 2 * np.pi) / (2 ** (j + 1))
    qc.p(angle, qreg[j])

# Apply inverse QFT
qc.append(QFT(num_qubits=NUMBITS, do_swaps=False).inverse(), qreg)

# Measure
qc.measure(qreg, creg)

# Run simulation
backend = AerSimulator()
job = backend.run(transpile(qc, backend), shots=1024)
counts = job.result().get_counts()

# Print results
print("\n--- MEASUREMENT COUNTS ---")
for k, v in sorted(counts.items(), key=lambda x: -x[1]):
    print(f"{k} : {v}")

# Interpret output (2's complement to int)
most_common = max(counts, key=counts.get)
result_int = int(most_common, 2)
if result_int >= (1 << (NUMBITS - 1)):
    result_int -= (1 << NUMBITS)

print(f"\nMost common result: {most_common} (interpreted as {result_int})")
qc.draw()

Input a (quantum):   -12 -> 2's comp bits (LSB first): [0, 0, 1, 0, 1]
Input b (classical): -1 -> 2's comp bits (LSB first): [1, 1, 1, 1, 1]

--- MEASUREMENT COUNTS ---
10011 : 1024

Most common result: 10011 (interpreted as -13)


┌──────┐ ┌───────┐  ┌───────┐┌─┐            
q_0: ─────┤0     ├─┤ P(-π) ├──┤0      ├┤M├────────────
          │      │┌┴───────┴┐ │       │└╥┘┌─┐         
q_1: ─────┤1     ├┤ P(-π/2) ├─┤1      ├─╫─┤M├─────────
     ┌───┐│      │├─────────┤ │       │ ║ └╥┘┌─┐      
q_2: ┤ X ├┤2 QFT ├┤ P(-π/4) ├─┤2 IQFT ├─╫──╫─┤M├──────
     └───┘│      │├─────────┤ │       │ ║  ║ └╥┘┌─┐   
q_3: ─────┤3     ├┤ P(-π/8) ├─┤3      ├─╫──╫──╫─┤M├───
     ┌───┐│      │├─────────┴┐│       │ ║  ║  ║ └╥┘┌─┐
q_4: ┤ X ├┤4     ├┤ P(-π/16) ├┤4      ├─╫──╫──╫──╫─┤M├
     └───┘└──────┘└──────────┘└───────┘ ║  ║  ║  ║ └╥┘
c: 5/═══════════════════════════════════╩══╩══╩══╩══╩═
                                        0  1  2  3  4